<a href="https://colab.research.google.com/github/burnout13l/Tesis_R-V/blob/main/datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#.JSON TODO

In [ ]:
!pip install pdfplumber pandas openpyxl
import pdfplumber
import os
import re
import json

carpeta = "certificados"  #ruta en Colab

certificados_datos = {}

for archivo in os.listdir(carpeta):
    if archivo.lower().endswith(".pdf"):
        ruta_pdf = os.path.join(carpeta, archivo)
        with pdfplumber.open(ruta_pdf) as pdf:
            texto_completo = ""
            for page in pdf.pages:
                texto_completo += page.extract_text() + "\n"

            # Diccionario con las variables en orden
            datos = {
                "cliente": None,
                "fecha_calibracion": None,
                "periodicidad": None,
                "codigo": None,
                "instrumento": None,
                "ubicación_instrumento": None,
                "marca": None,
                "modelo": None,
                "unidad": None,
                "rango": None,
                "division_escala": None,
                "lugar_calibracion": None,
                "temperatura": None,
                "humedad": None,
            }

            # === Regex para cada campo ===
            cliente = re.search(r"Cliente.*?:\s*(.*)\s+Certificado", texto_completo)
            datos["cliente"] = cliente.group(1).lstrip("* ").strip() if cliente else None

            fecha = re.search(r"Fecha de inicio y final de Calibración:\s*(.*)", texto_completo)
            datos["fecha_calibracion"] = fecha.group(1).strip() if fecha else None

            periodicidad = re.search(r"Periodicidad.*?:\s*(.*)", texto_completo)
            datos["periodicidad"] = periodicidad.group(1).lstrip("* ").strip() if periodicidad else None

            codigo = re.search(r"Código[:\*]?\s*(.*?)\n", texto_completo)
            datos["codigo"] = codigo.group(1).lstrip("* ").strip() if codigo else None

            instrumento = re.search(r"Denominación[:\*]?\s*(.*?)\n", texto_completo)
            datos["instrumento"] = instrumento.group(1).strip() if instrumento else None

            ubicacion = re.search(r"Ubicación[:\*]?\s*(.*?)\n", texto_completo)
            datos["ubicación_instrumento"] = ubicacion.group(1).lstrip("* ").strip() if ubicacion else None

            marca = re.search(r"Marca[:\*]?\s*(.*?)\n", texto_completo)
            datos["marca"] = marca.group(1).replace("Marca:", "").strip() if marca else None

            modelo = re.search(r"Modelo[:\*]?\s*(.*?)\n", texto_completo)
            datos["modelo"] = modelo.group(1).lstrip("* ").replace("Modelo:", "").strip() if modelo else None

            # === Rango completo (con unidad) ===
            rango = re.search(r"Rango[:\*]?\s*(.*?)\n", texto_completo)
            datos["rango"] = rango.group(1).replace("Rango:", "").strip() if rango else None

            # === División de Escala (número + unidad juntos) ===
            div_escala = re.search(r"División de Escala[:\*]?\s*(.*?)\n", texto_completo)
            datos["division_escala"] = div_escala.group(1).strip() if div_escala else None

            # === Unidad (extraída desde División de Escala) ===
            div_match = re.search(r"División de Escala:\s*[\d,\.]+\s*([A-Za-z%µ/²]+)", texto_completo)
            datos["unidad"] = div_match.group(1) if div_match else None

            # === Lugar de Calibración ===
            lugar = re.search(r"Lugar de calibraci[oó]n[:\*]?\s*(.*?)\n", texto_completo)
            datos["lugar_calibracion"] = lugar.group(1).strip() if lugar else None

            # === Temperatura y Humedad separadas ===
            th_match = re.search(r"([\d]+(?:[.,]\d+)?\s*±?\s*[\d,\.]*\s*ºC)\s*/\s*([\d]+(?:[.,]\d+)?\s*±?\s*[\d,\.]*\s*%RH)", texto_completo)
            if th_match:
                datos["temperatura"] = th_match.group(1)
                datos["humedad"] = th_match.group(2)

            # === Extraer tabla de resultados ===
            tabla_match = re.search(r"RESULTADOS DE LA CALIBRACIÓN(.*?)(CURVA DE ERRORES DE MEDICIÓN|$)", texto_completo, re.S)
            tabla_filas = {}
            fila_contador = 1

            if tabla_match:
                tabla_texto = tabla_match.group(1).strip()
                filas = [line.strip() for line in tabla_texto.split("\n") if line.strip()]

                for fila in filas:
                    # === Conservar solo la fila de unidades y filas numéricas o con ± ===
                    if re.search(r"[A-Za-z()]", fila):
                        # Conservar la fila de unidades si contiene la unidad principal
                        if datos["unidad"] and datos["unidad"] in fila:
                            tabla_filas[f"Fila {fila_contador}"] = fila
                            fila_contador += 1
                        continue  # saltar todas las demás filas con letras

                    # Extraer todos los valores con ±
                    valores_pm = re.findall(r"±\s*[\d,\.]+", fila)

                    # Separar fila original: quitar los valores con ±
                    fila_sin_pm = re.sub(r"±\s*[\d,\.]+", "", fila).strip()

                    # Si la fila sin ± todavía tiene números, la guardamos
                    if re.search(r"[\d]", fila_sin_pm):
                        tabla_filas[f"Fila {fila_contador}"] = fila_sin_pm
                        fila_contador += 1

                    # Si hay valores con ± y la fila no estaba solo con ±, creamos fila nueva
                    if valores_pm and not re.fullmatch(r"(?:±\s*[\d,\.]+\s*)+", fila):
                        tabla_filas[f"Fila {fila_contador}"] = " ".join(valores_pm)
                        fila_contador += 1

                    # Si la fila ya era solo ±, la dejamos tal cual y no duplicamos
                    if valores_pm and re.fullmatch(r"(?:±\s*[\d,\.]+\s*)+", fila):
                        tabla_filas[f"Fila {fila_contador}"] = fila
                        fila_contador += 1

            datos["Tabla Resultados"] = tabla_filas

            certificados_datos[archivo] = datos

# === Guardar todo en JSON ===
with open("certificados_datos_con_tabla.json", "w", encoding="utf-8") as f:
    json.dump(certificados_datos, f, ensure_ascii=False, indent=4)

print(f"Se procesaron {len(certificados_datos)} PDFs y se generó 'certificados_datos_con_tabla.json'.")

Se procesaron 26 PDFs y se generó 'certificados_datos_con_tabla.json'.


**convertir a excel todo**

In [ ]:
import json
import pandas as pd
import re

# === 1. Cargar el JSON generado antes ===
with open("certificados_datos_con_tabla.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# === 2. Lista para registros en formato tabular ===
registros = []

for archivo, valores in data.items():
    # Extraer metadatos
    cliente = valores.get("cliente")
    fecha_calibracion = valores.get("fecha_calibracion")
    periodicidad = valores.get("periodicidad")
    codigo = valores.get("codigo")
    instrumento = valores.get("instrumento")
    ubicacion = valores.get("ubicación_instrumento")
    marca = valores.get("marca")
    modelo = valores.get("modelo")
    unidad = valores.get("unidad")
    rango = valores.get("rango")
    division_escala = valores.get("division_escala")
    lugar = valores.get("lugar_calibracion")
    temperatura = valores.get("temperatura")
    humedad = valores.get("humedad")

    tabla = valores.get("Tabla Resultados", {})

    # === Buscar el primer valor con ± para incertidumbre ===
    incertidumbre_valor = None
    for fila in tabla.values():
        if "±" in fila:
            match = re.search(r"±\s*[\d,.]+", fila)
            if match:
                incertidumbre_valor = match.group(0)
                break

    # === Localizar fila de unidades ===
    fila_unidades = None
    for fila in tabla.values():
        if re.search(r"[A-Za-z%µ/²]", fila):
            fila_unidades = fila.split()
            break

    if not fila_unidades:
        continue  # si no encontramos fila de unidades, saltar este archivo

    # Posiciones de interés
    try:
        primer_kpa_idx = fila_unidades.index("kPa")  # primer KPa (o usar unidad principal)
        segundo_kpa_idx = len(fila_unidades) - 4   # penúltimo kPa, siempre 4 antes del final
        # indices calculados según tus reglas
        idx_patron = primer_kpa_idx + 1
        idx_calibrado = segundo_kpa_idx - 3
        idx_error = segundo_kpa_idx - 2
    except ValueError:
        continue

    # === Procesar filas numéricas ===
    for _, fila in tabla.items():
        if "±" in fila:
            continue  # saltar fila de incertidumbre
        # ignorar filas que sean encabezados
        if re.search(r"[A-Za-z()]", fila):
            continue

        partes = fila.split()
        # asegurarnos de no exceder índice
        if max(idx_patron, idx_calibrado, idx_error) >= len(partes):
            continue

        punto_patron = partes[idx_patron]
        punto_calibrado = partes[idx_calibrado]
        error = partes[idx_error]

        registros.append({
            "archivo": archivo,
            "cliente": cliente,
            "fecha_calibracion": fecha_calibracion,
            "periodicidad": periodicidad,
            "codigo": codigo,
            "instrumento": instrumento,
            "ubicación_instrumento": ubicacion,
            "marca": marca,
            "modelo": modelo,
            "unidad": unidad,
            "rango": rango,
            "division_escala": division_escala,
            "lugar_calibracion": lugar,
            "temperatura": temperatura,
            "humedad": humedad,
            "punto_patron": punto_patron,
            "punto_calibrado": punto_calibrado,
            "error": error,
            "incertidumbre": incertidumbre_valor
        })

# === 3. Crear DataFrame final ===
df = pd.DataFrame(registros)

# === 4. Exportar a Excel y CSV ===
df.to_excel("resultados_calibracion_tabla.xlsx", index=False)
df.to_csv("resultados_calibracion_tabla.csv", index=False)

print("Datos exportados a 'resultados_calibracion_tabla.xlsx' y 'resultados_calibracion_tabla.csv'")

Datos exportados a 'resultados_calibracion_tabla.xlsx' y 'resultados_calibracion_tabla.csv'
